In [1]:
import requests
from datetime import datetime
import sqlite3 # fot sqlute database support
from bs4 import BeautifulSoup

In [ ]:
class Crawler:
    pttUrl = "https://www.ptt.cc"

    def __init__(self, board):
        self.board = board
        self.Articles = []
        self.boardUrl = "/bbs/{}".format(board)
        self.contents = []
#         request = requests.get(Crawler.pttUrl + self.boardUrl, cookies = {"over18": "1"})
#         if request.status_code == 404:
#             print("No such board")
#             return
#         pageText = crawler.pageText.split("r-list-sep")[0]
#         self.pageText = request.text
        
    def getToday(self, date=None, pageNext=None, keyword=None, num_posts=100):
        if pageNext == None:
            pageNext = self.boardUrl
        request = requests.get(Crawler.pttUrl + pageNext, cookies = {"over18": "1"})#滿18歲
        if request.status_code == 404:
            print("No such board")
            return
        self.pageText = request.text
        soup = BeautifulSoup(self.pageText, "lxml")
        pageNext = soup.find("div", "btn-group btn-group-paging").find_all("a")[1].attrs["href"]
        if date == None:
            date = datetime(2020, 2, 12) #日期
        pageText = self.pageText.split("r-list-sep")[0]
        soup = BeautifulSoup(pageText, "lxml")
        for post in soup.select("div.r-ent"):
            url = post.find("div", "title").a
            if url == None:
                continue
            else:
                url = post.find("div", "title").a.attrs["href"]
            articleTxt = requests.get(Crawler.pttUrl + url, cookies = {"over18": "1"}).text
            articleSoup = BeautifulSoup(articleTxt, "lxml")
            title = articleSoup.find("title").text  # 標題
            content = articleSoup.find("meta", property="og:description").get("content")  # 文章內容
            if title == None or content == None:
                continue
            if keyword != None:
                if title.find(keyword) < 0 and content.find(keyword) < 0:
                    continue
            print(Crawler.pttUrl + url)
#             print(type(title))
            self.contents.append(title + ":" + content)
#             self.contents.append(content)
            push_list = []
            for push in articleSoup.find_all("span", class_="push-content"):
                if len(push.text[2:]) <= 0:
                    continue
                push_list.append(push.text[2:])  # 推文
            self.contents.append(push_list)  
            time_str = post.find("div", "date").text.strip()
            time_obj = datetime.strptime(time_str+"/2020", '%m/%d/%Y')
            #if True:  # 一篇文章
            if time_obj < date:  # 不是今天的文章
                return
        else:
            if len(self.contents)/2 > num_posts:
                return
            print(pageNext)
            self.getToday(date, pageNext, keyword, num_posts)

In [ ]:
board = "FoodDelivery" #FoodDelivery版
crawler = Crawler(board)
crawler.getToday(num_posts=10)
# crawler.getToday(keyword="陳時中", num_posts=10)

In [ ]:
import urllib.request as req
url="https://www.ptt.cc/bbs/FoodDelivery/index.html"

#建立一個request物件，附加request headers資訊
request=req.Request(url,headers={
    "user-agent":"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Mobile Safari/537.36"
}) 
#模擬一般使用者User-Agent

with req.urlopen(request) as response:
    data=response.read().decode("utf-8")
# print(data)

import bs4
root=bs4.BeautifulSoup(data,"html.parser")
print(root.title.string) #抓頁面標題


root1=bs4.BeautifulSoup(data,"html.parser")
titles=root1.find_all("div",class_="title") #尋找class="title"的div標籤
for title in titles: #利用迴圈取出整個頁面文章標題
    if title.a!=None:
        print(title.a.string)

In [ ]:
import requests
from bs4 import BeautifulSoup
url="https://www.ptt.cc/bbs/FoodDelivery/index.html"
List = []
def get_all_href(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    results = soup.select("div.title")
    for item in results:
        a_item = item.select_one("a")
        title = item.text
        if a_item:
            print(title, 'https://www.ptt.cc'+ a_item.get('href'))
#             List+='https://www.ptt.cc'+ a_item.get('href')
            
        
for page in range(0,1):
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"html.parser")
    btn = soup.select('div.btn-group > a')
    up_page_href = btn[3]['href'] 
    next_page_url = 'https://www.ptt.cc' + up_page_href
    url = next_page_url
    get_all_href(url = url)
    
# List[0]

In [2]:
import bs4
import requests
from bs4 import BeautifulSoup
import requests as rq
from bs4 import BeautifulSoup
setSQLData = []
url="https://www.ptt.cc/bbs/FoodDelivery/index.html"
List = []
def get_all_href(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    results = soup.select("div.title")
    for item in results:
        a_item = item.select_one("a")
        title = item.text
        if a_item:
            print(title, 'https://www.ptt.cc'+ a_item.get('href'))
            url ='https://www.ptt.cc'+ a_item.get('href')
            response = rq.get(url) # 用 requests 的 get 方法把網頁抓下來
            soup = bs4.BeautifulSoup(response.text,"html.parser")

            ## PTT 上方4個欄位
            header = soup.find_all('span','article-meta-value')
            articles = soup.find_all('div', 'push')

            # 作者
            author = header[0].text
            # 看版
            board = header[1].text
            # 標題
            title = header[2].text
            # 日期
            date = header[3].text


            ## 查找所有html 元素 抓出內容
            main_container = soup.find(id='main-container')
            # 把所有文字都抓出來
            all_text = main_container.text
            # 把整個內容切割透過 "-- " 切割成2個陣列
            pre_text = all_text.split('--')[0]

            # 把每段文字 根據 '\n' 切開
            texts = pre_text.split('\n')
            # 如果你爬多篇你會發現 
            contents = texts[2:]
            # 內容
            content = '\n'.join(contents)


            # 顯示
            print('作者：'+author)
            print('看板：'+board)
            print('標題：'+title)
            print('日期：'+date)
            print('內容：'+content) 
            print('------------------------------------------')
            
        for article in articles:
        #去除掉冒號和左右的空白
            
            messages = article.find('span','f3 push-content').getText().replace(':','').strip()
            print('messages:'+messages)
            List.append(messages)
            print('----------------------------------------------------') 
        row = [author, board, title, date, content] 
        setSQLData.append(row)#將每條數據再次寫入列表 
        
        
for page in range(0,10):
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"html.parser")
    btn = soup.select('div.btn-group > a')
    up_page_href = btn[3]['href'] 
    next_page_url = 'https://www.ptt.cc' + up_page_href
    url = next_page_url
    get_all_href(url = url)
    
print(List)
len(List)
# print(messages)
   
wb = Workbook()

# 設置Excel文件名
dest_filename = 'ptt.xlsx'
# 新建一個表 
ws1 = wb.active 
# 設置表頭 
titleList = ['作者', '看板', '標題', '日期', '內容']
for row in range(len(titleList)): 
    c = row + 1 
    ws1.cell(row=1, column=c, value=titleList[row])
# 填寫表內容 
for listIndex in range(len(setSQLData)):
    ws1.append(setSQLData[listIndex])
wb.save(filename=dest_filename)
print(List)# 所有外送版的留言
len(List) 


Re: [問題] 熊貓又掛了？
 https://www.ptt.cc/bbs/FoodDelivery/M.1623900143.A.5B8.html
作者：TaiwanDPP (偉)
看板：FoodDelivery
標題：Re: [問題] 熊貓又掛了？
日期：Thu Jun 17 11:22:21 2021
內容：熊貓掛了

但是飢餓讓我找到了辦法
https://i.imgur.com/bVlxLAG.jpg

1.用電腦版登入

2.去google搜尋你要吃的店面
的foodpanda網址

https://i.imgur.com/jGlftBo.jpg

3.正常點餐

我猜app在已訂餐點按重新點餐也可以

※ 引述《medama ( )》之銘言：
: http://i.imgur.com/tGTWUlF.jpg
: 第一次看到這種情況
: 如果下大雨就算了
: 可是現在台北出大太陽耶
: 而且才11點而已


------------------------------------------
messages:從歷史訂單也可連到之前訂過的店家點餐
----------------------------------------------------
messages:剛剛用電腦版的foodpanda
----------------------------------------------------
messages:有一陣子可以找店家訂餐
----------------------------------------------------
messages:不過現在不行了
----------------------------------------------------
messages:付款會失敗QQ
----------------------------------------------------
messages:去電腦版，底下會有一排城市可以選
----------------------------------------------------
messages:選你在的城市
----------------------------------------------------
messages:就可以找到店家

作者：caspar1213 (caspar)
看板：FoodDelivery
標題：[抱怨] foodpanda跟店家真蝦
日期：Mon Jun 21 10:20:55 2021
內容：因為疫情大家都外送或自取

結果我人到現場沒開店

是整人嗎？

https://i.imgur.com/71mABgb.jpg
https://i.imgur.com/QFWv7ME.jpg



------------------------------------------
messages:店家忘了關
----------------------------------------------------
messages:想說，下雨天，不麻煩外送員，結果去，撲空
----------------------------------------------------
messages:傻眼
----------------------------------------------------
messages:因為熊貓的店家端沒開平板也是會上線的 要手動關
----------------------------------------------------
messages:FOODPANDA會自己接單. 所以我目前遇到都改用UBER EAT
----------------------------------------------------
messages:因為店家表示.忙不過來還接單.有事嗎.
----------------------------------------------------
messages:所以你很衰遇到平板沒關.熊貓幫你接單了 XD
----------------------------------------------------
messages:上次也是一樣遇到別家店，下單時間要到時，給你取消訂
----------------------------------------------------
messages:單，超傻眼，不就還有看到快重找別家店，熊貓只會說
----------------------------------------------------
messages:不好意思造成您的困擾。
--

作者：shinmin1226 (shin)
看板：FoodDelivery
標題：[心得] foodpanda 買一送一與pro優惠
日期：Sat Jun 26 16:13:17 2021
內容：剛在省錢板發文被刪文==
能幫到一個是一個吧
https://i.imgur.com/0CYECaZ.jpg

買一送一跟pro優惠不能同時使用
所以會員只能打九折

https://i.imgur.com/5TP2y25.jpg

https://i.imgur.com/gujGheH.jpg

所以大家想點的餐點
點下去了記得截圖
再去跟客服講吧

他們客服只會鬼打牆

https://i.imgur.com/TgBgJ7e.jpg


------------------------------------------
messages:之前討論過了 pro反而不能買一送一 真的很爛
----------------------------------------------------
messages:正常發揮
----------------------------------------------------
messages:謝謝分享
----------------------------------------------------
messages:反應過了，客服只會鬼打牆
----------------------------------------------------
messages:上次也是訂購後才發現這樣
----------------------------------------------------

[問題] UberEats「放在門口」一直被按計時？
 https://www.ptt.cc/bbs/FoodDelivery/M.1624705078.A.3E7.html
作者：telgniw (Yi)
看板：FoodDelivery
標題：[問題] UberEats「放在門口」一直被按計時？
日期：Sat Jun 26 18:57:55 2021
內容：已爬板＆上網搜尋都沒有找到相關資訊。


已經使用「放在門口」選項數月，大樓管理員都會迅速幫開門和通知餐點到達。

但是三級警戒開始會遇到外送員在按了「我已抵達」之後，

作者：sixmochida (Q_Q)
看板：FoodDelivery
標題：Re: [問題] Foooodpanda付款失敗
日期：Sat Jun 26 23:24:14 2021
內容：最近也發生一樣的問題，
換了很多卡，兩個帳號都無法刷卡，
只剩外送貨到付款功能。。。


※ 引述《Manas0816 ()》之銘言
: 早安！
: 早餐是開啟美好一天的一餐！
: Foodpanda 生鮮食品用信用卡支付被拒絕...。
: 有人也這樣嗎？謝謝！
: https://i.imgur.com/ATPgEUv.jpg



------------------------------------------

[問題] Uber Eats優饗跟折扣優惠是否有衝突
 https://www.ptt.cc/bbs/FoodDelivery/M.1624791496.A.9BA.html
作者：qeerasfolk (平凡克)
看板：FoodDelivery
標題：[問題] Uber Eats優饗跟折扣優惠是否有衝突
日期：Sun Jun 27 18:58:14 2021
內容：大家好。

不好意思，還沒用過優饗，不知道199免運費是不是需要佔用優惠折扣的條件，還是像最
近生鮮打7折也可以疊加使用呢？

再麻煩各位有訂閱的達人解答了！

謝謝~



------------------------------------------
messages:可疊
----------------------------------------------------
messages:謝謝
----------------------------------------------------
messages:可疊 但是要注意缺貨問題
----------------------------------------------------
messages:如果因為缺貨 導致最後金額未滿199 還是會收運費
----------------------------------------------------
messages:就這點來說熊貓的pro比較好
----------------------------------------------------
messag

作者：CEO2B (Dream Chaser)
看板：FoodDelivery
標題：熊貓縮小外送範圍？
日期：Sun Jun  6 17:11:18 2021
內容：
疫情以來很依賴外送，甚至加了 pandapro

但今天發現以往很多能選的店家全部顯示關閉

共同點是都在市區（我住郊區）

看了客服問答，可能是因為熊貓縮小外送範圍的緣故

第一次知道還有這情況，大家有遇過嗎？

如果太常遇到，可能會退訂 pandapro 了

------------------------------------------
messages:昨天開始遇到
----------------------------------------------------
messages:之前訂得到的店家，加pro都不見了！
----------------------------------------------------
messages:外送員太少店家會被鎖區只有附近能訂
----------------------------------------------------
messages:某些知名連鎖店（麥當勞必勝客那種）好像不會被鎖區
----------------------------------------------------
messages:難怪，我北投區能訂的東西也變超少
----------------------------------------------------
messages:郊區雨天根本訂不到 = =
----------------------------------------------------
messages:兩間都一樣啊  雨下到一個程度就不再是加多少錢會有
----------------------------------------------------
messages:司機願意跑的問題了
----------------------------------------------------
messages:所以縮區→關閉訂餐都是對平台避免訂單沒人送的保護措施
----------------------------------------------------
messages:何況你還是付跟原本

...

作者：chocoball (巧克力球)
看板：FoodDelivery
標題：[新聞] 熊貓「全系列」粉紅曝光　狂粉：我生病了
日期：Mon Feb  1 19:41:22 2021
內容：熊貓「全系列」粉紅曝光　狂粉：我生病了


生活中心／詹鎰睿報導

台灣懶人外送風氣正夯，各式各樣的平台互相競爭，也各自擁有粉絲力挺。對此，一名
Foodpanda熊貓的瘋狂粉絲就收集了一堆熊貓系列的週邊商品，結果戰果照曝光後，眾人
全部看傻，「也太可愛了吧！」


臉書社團「外送員的奇聞怪事」，有位網友PO文自嘲「我一點都不想知道我484生病了！
」可以從壯觀的照片看到，攤在地上的全都是印有熊貓的商品，包括從小到大的外送包包
、熊貓外送紙袋、熊貓保暖杯、熊貓小傘、熊貓餐具、熊貓外送帶、熊貓口罩和牌子等等
，幾乎所有熊貓出過的，她全都收集到了。

成果照一出，不少同行紛紛震驚「我看到粉紅色會怕」、「Food panda,應該頒個獎章給
你」、「終身正職外送職人」、「外送賺的錢，又給熊貓了」、「這麼熱愛公司」、「快
去看醫生吧」；不過也有粉絲敲碗，「想知道去哪裡可以買到呢？」「嚴重熊貓控」、「
熊貓營業中的掛牌要怎麼拿啊我想拿一個來掛車上」。




對此，原PO則透露，有一大部分是從之前新北耶誕城實體快閃店買來的，另外，有些商品
也可以從熊貓超市取得，至於熊貓營業中掛牌「那是朋友去跟店家凹來的」、「取餐臨停
是自家設計的非賣品」此外，也有網友曬出其他商品，包括安全帽、以前絕版的橘色外送
包、第一代外送包等等，掀起眾人熱議。



https://www.setn.com/News.aspx?NewsID=890613  三立




------------------------------------------

[新聞] 魂斷路口！貨車疑跨雙黃線 熊貓外送員撞
 https://www.ptt.cc/bbs/FoodDelivery/M.1612179821.A.CAB.html
作者：chocoball (巧克力球)
看板：FoodDelivery
標題：[新聞] 魂斷路口！貨車疑跨雙黃線 熊貓外送員撞
日期：Mon Feb  1 19:43:38 2021
內容：魂斷路口！貨車疑跨雙黃線 熊貓外送員撞上亡


施協源,吳姝叡 王皓慈,許猛捷
2021年2月1日 週一 

作者：medama ( )
看板：FoodDelivery
標題：[優惠] 熊貓超市嘉義店開幕 限時5折
日期：Thu Jan  7 11:47:01 2021
內容：https://www.facebook.com/foodpandaTaiwan/posts/3702022629917225
2021/1/13前，上pandamart 熊貓超市嘉義店，輸入優惠碼【27DS50】，享超殺5折優惠

*活動期間為2021/1/7至2021/1/13，限訂購pandamart 熊貓超市嘉義店且需於每日晚上
8-10點方可使用，每人限用1次，每日限量50組，最高折抵$200，優惠碼數量有限兌完為
止。活動詳情請以foodpanda app及官網為主，foodpanda保留隨時單方修改權利。


------------------------------------------

[討論] 熊貓版優饗？
 https://www.ptt.cc/bbs/FoodDelivery/M.1610006038.A.D68.html
作者：popy8789 (面對它)
看板：FoodDelivery
標題：[討論] 熊貓版優饗？
日期：Thu Jan  7 15:53:56 2021
內容：
大家都知道UberEats訂120可以享用199免運

但現在也是限縮為有綠色票券的才免運

https://i.imgur.com/5vA1iq6.jpg

現在熊貓也開始準備推訂閱制

名稱叫panda pro

但我看來似乎是個比較爛版本的優饗

還要訂到229才會免運

https://i.imgur.com/cFZkrJJ.jpg

唯一不同的是月費似乎是比較低，但用原價就只比UberEats少21元而已

而且Uber還有一個雙饗含計程車打95折的

大家看好嗎還是跟我一樣覺得不看好

一餐能吃到229多人不多吧

------------------------------------------
messages:公司一起訂的話感覺不錯 不知道限時五折有哪些
----------------------------------------------------
messages:如果限時五折的餐廳夠好我會想付月費看看
-----------------------------

作者：nyu5765 (nyu5765)
看板：FoodDelivery
標題：[討論] 外送推薦的生財工具
日期：Mon Jan 18 20:26:18 2021
內容：請問一下

假如要做外送

有比較推薦哪類的車充.貨架另外還有防雨的手機支架但是可以插充電的

另外台北市有比較推薦的機車停車格嗎(收費便宜的)



------------------------------------------
messages:感覺台北市是地獄難度  難怪錢多外送費比較高
----------------------------------------------------
messages:台北市可以買月票 一個月300
----------------------------------------------------

[心得] Ubereats 運費與服務距離
 https://www.ptt.cc/bbs/FoodDelivery/M.1611041017.A.18A.html
作者：orz65535 (′‧ω‧‵)
看板：FoodDelivery
標題：[心得] Ubereats 運費與服務距離
日期：Tue Jan 19 15:23:35 2021
內容：目前設定的運費三階

台北市、新北市

25 , 40 , 60


其餘區域

15 , 30 , 60

不同區域狀態可能不同，這裡以我所在的台中來說明

服務費原則上是看距離，但部分店家可以容許該運費延伸較遠距離(具體機制不明)

絕大多數15運費距離家裡最遠約1.5KM，但少數店家可以遠到2.5KM

60運費也不是每個店家能點的到，因為不同店家可送最遠範圍也不同

實際送過最遠是5.5公里(事後用google測量，不是系統內建亂繞的導航)

但點餐可能多數店家超過3.5公里就點不到了


後來仔細分析，那些可以送很遠的通常也是UE的紅店，以台中來說像偈亭、老向的店

都是常常送超遠的大紅店(這些店也是常常被拿掉優饗免運標籤的店)



麥當勞部分，因為合作分店非常多，但系統有時候優先排序的不一定是你附近的店

這就導致常常特定門市取餐卻送2-3公里遠的住家(該住家當地也是有合作的麥當勞)

也許這跟用優饗方案後看不到運費所以看到就點有關就是了

另外UE主要只顯示門市名稱，像新大雅店(實際

作者：medama ( )
看板：FoodDelivery
標題：[公告] FoodDelivery版暫行版規
日期：Mon Dec 28 15:03:34 2020
內容：0.
發表文章請遵守網路討論禮節，
如文章內容過於簡短無意義，
或是擾亂版面秩序或嚴重爭吵，則由板主視情況刪文處理。
若判定為惡意鬧版，板主可視情況水桶禁言或退文。

1.
個人邀請碼僅限在[自介]文分享（[自介]文需按照內建格式填寫）
違者刪除。
發表自介文每人每年限一篇，
例：2021年1月1日發自介文，下一篇自介文需等2022年1月2日才能發

2.
禁止廣告文
外送店家可發廣告文，每店每年限一篇。
違者刪除。

以上為暫訂版規，今後將隨時依照需求增補修改。


------------------------------------------

[新聞] 屏東第一起 美食外送員自撞電桿身亡
 https://www.ptt.cc/bbs/FoodDelivery/M.1609139936.A.D76.html
作者：medama ( )
看板：FoodDelivery
標題：[新聞] 屏東第一起 美食外送員自撞電桿身亡
日期：Mon Dec 28 15:18:54 2020
內容：https://www.cna.com.tw/news/asoc/202012280128.aspx

屏東第一起 美食外送員自撞電桿身亡
最新更新：2020/12/28 14:06

（中央社記者郭芷瑄屏東縣28日電）屏東市24日發生一名美食外送員自撞電線桿死亡事件
，屏東警分局今天表示，已調閱現場監視器查原因。這是屏東縣第一起外送員車禍死亡意
外。

警方表示，26歲的吳姓外送員24日在屏東市金城街220-1號前，碰撞路燈燈桿後倒地，被
路人通報送醫急救，吳姓外送員被診斷腦出血，隔日凌晨宣告不治。

警方表示，已調閱現場及周邊路口監視器資料，肇事原因尚待調查釐清中。

屏東縣今年已發生多起外送員交通事故，都只是受傷，這是第一起外送員車禍意外死亡事
件，今年外送員車禍意外事件到底有多少件，屏東縣警察局交通隊表示，沒有確切的數據
。

交通隊表示，截至17日，屏東縣今年A1（24小時內死亡）交通事故126件、造成128人死亡
，較去（108）年同期107件、死亡108人，增加19件、20人。A2類交通事故（受

NameError: name 'Workbook' is not defined

In [ ]:
len(List) 

In [ ]:
import requests as rq
from bs4 import BeautifulSoup
from openpyxl import Workbook
setSQLData = []
url = "https://www.ptt.cc/bbs/FoodDelivery/M.1624021156.A.08E.html" # PTT NBA 板
response = rq.get(url) # 用 requests 的 get 方法把網頁抓下來
soup = bs4.BeautifulSoup(response.text,"html.parser")

## PTT 上方4個欄位
header = soup.find_all('span','article-meta-value')

# 作者
author = header[0].text
# 看版
board = header[1].text
# 標題
title = header[2].text
# 日期
date = header[3].text


## 查找所有html 元素 抓出內容
main_container = soup.find(id='main-container')
# 把所有文字都抓出來
all_text = main_container.text
# 把整個內容切割透過 "-- " 切割成2個陣列
pre_text = all_text.split('--')[0]
    
# 把每段文字 根據 '\n' 切開
texts = pre_text.split('\n')
# 如果你爬多篇你會發現 
contents = texts[2:]
# 內容
content = '\n'.join(contents)


# 顯示
print('作者：'+author)
print('看板：'+board)
print('標題：'+title)
print('日期：'+date)
print('內容：'+content)

row = [author, board, title, date, content] 
setSQLData.append(row)#將每條數據再次寫入列表


wb = Workbook()

# 設置Excel文件名
dest_filename = 'ptt.xlsx'
# 新建一個表 
ws1 = wb.active 
# 設置表頭 
titleList = ['作者', '看板', '標題', '日期', '內容']
for row in range(len(titleList)): 
    c = row + 1 
    ws1.cell(row=1, column=c, value=titleList[row])
# 填寫表內容 
for listIndex in range(len(setSQLData)):
    ws1.append(setSQLData[listIndex])
wb.save(filename=dest_filename)

setSQLData[0]

In [ ]:
import requests as rq
from bs4 import BeautifulSoup
url = "https://www.ptt.cc/bbs/FoodDelivery/M.1625553790.A.461.html" 
response = rq.get(url) # 用 requests 的 get 方法把網頁抓下來
soup = bs4.BeautifulSoup(response.text,"html.parser")

#使用find_all()找尋特定目標
articles = soup.find_all('div', 'push')
#印出
# print(articles)
#寫入檔案中
# with open('movie_message.txt','w') as f:
for article in articles:
        #去除掉冒號和左右的空白
    messages = article.find('span','f3 push-content').getText().replace(':','').strip()
    print('messages:'+messages)
    print('----------------------------------------------------')
#     f.write(messages + "\n")

In [3]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import jieba

In [4]:
with open("text1.csv", encoding="big5", errors='ignore') as f:
    text = f.read()
text
# 所有ptt外送版的文章

'"熊貓掛了\n\n但是飢餓讓我找到了辦法\nhttps://i.imgur.com/bVlxLAG.jpg\n\n1.用電腦版登入\n\n2.去google搜尋你要吃的店面\n的foodpanda網址\n\nhttps://i.imgur.com/jGlftBo.jpg\n\n3.正常點餐\n\n我猜app在已訂餐點按重新點餐也可以\n\n※ 引述《medama ( )》之銘言：\n: http://i.imgur.com/tGTWUlF.jpg\n: 第一次看到這種情況\n: 如果下大雨就算了\n: 可是現在台北出大太陽耶\n: 而且才11點而已\n\n"\n"\n如題，有訂閱pro卻還收取運費？\n熊貓最近是怎麼了！！！\n\nhttps://i.imgur.com/0xqr49k.jpg\n被收運費（價錢會是280+30，因為買一送一）\nhttps://i.imgur.com/aYpxAMN.jpg\n但我有訂閱\n"\n"各位好，\n\n我是台北大學社會系張毓芬助理教授的學生，這裡協助教授轉發「誰是外送員？台北、北京、武漢、新加坡、香港五個城市的比較研究」的研究計畫。目前台\n北組決定從大學生外送員開始做起。我們正在尋找有外送工作經驗的受訪者，不管是正在做\n的、曾經做過的，又願意接受訪談的，都非常歡迎發信填寫表單報名。我們會跟你聯繫，以您方便的時間與地點進行\n訪談，並提供全職外送員500車馬費，兼職外送員300元車馬費。如果你擔心疫情，我們也可以電訪或依照你希望的方式進行訪談。\n為了確保我們對你的談話內容有正確的理解，我們會進行錄音，並且會將錄音轉成逐字稿\n。所有訪談內容百分之百保密，使用資料時也會完全匿名。\n\n謝謝大家！疫情期間，希望大家一切平安！\n\n表單連結:https://bit.ly/3gDwxuu\n\n如有違反板規，還請版主提醒\n\n"\n"請問uber生鮮每週可以領取一次七五折的優惠碼\n\n那是我上週用完一次後，這週領取新的也不能用是嗎？剛剛用都無法使用\n\n不太懂，所謂的一次，是指永遠就這一次75折嗎？\n\n謝謝答覆\n\n"\n"我知道外送員分秒必爭，\n\n差了那幾分鐘的時間可能就會少好幾筆的單，\n\n敬愛的外送員辛苦了。\n\n重點我還是要提醒一下，\n\n要趕單我知道也非常理解，\n\n您們從商家出來開始

In [5]:
# all=text+List
# print(List)
type(text)

str

In [6]:
import re
sub_text = re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a])","",text)
print(sub_text)
# 文章去標點、去多餘字詞

熊貓掛了但是飢餓讓我找到了辦法httpsiimgurcombVlxLAGjpg1用電腦版登入2去google搜尋你要吃的店面的foodpanda網址httpsiimgurcomjGlftBojpg3正常點餐我猜app在已訂餐點按重新點餐也可以引述medama之銘言httpiimgurcomtGTWUlFjpg第一次看到這種情況如果下大雨就算了可是現在台北出大太陽耶而且才11點而已如題有訂閱pro卻還收取運費熊貓最近是怎麼了httpsiimgurcom0xqr49kjpg被收運費價錢會是28030因為買一送一httpsiimgurcomaYpxAMNjpg但我有訂閱各位好我是台北大學社會系張毓芬助理教授的學生這裡協助教授轉發誰是外送員台北北京武漢新加坡香港五個城市的比較研究的研究計畫目前台北組決定從大學生外送員開始做起我們正在尋找有外送工作經驗的受訪者不管是正在做的曾經做過的又願意接受訪談的都非常歡迎發信填寫表單報名我們會跟你聯繫以您方便的時間與地點進行訪談並提供全職外送員500車馬費兼職外送員300元車馬費如果你擔心疫情我們也可以電訪或依照你希望的方式進行訪談為了確保我們對你的談話內容有正確的理解我們會進行錄音並且會將錄音轉成逐字稿所有訪談內容百分之百保密使用資料時也會完全匿名謝謝大家疫情期間希望大家一切平安表單連結httpsbitly3gDwxuu如有違反板規還請版主提醒請問uber生鮮每週可以領取一次七五折的優惠碼那是我上週用完一次後這週領取新的也不能用是嗎剛剛用都無法使用不太懂所謂的一次是指永遠就這一次75折嗎謝謝答覆我知道外送員分秒必爭差了那幾分鐘的時間可能就會少好幾筆的單敬愛的外送員辛苦了重點我還是要提醒一下要趕單我知道也非常理解您們從商家出來開始跑單可以麻煩外送員也要注意一下路況今天差一點點就撞到外送員了從商家騎車硬切出來也沒注意左右來車這樣騎法真的很危險啊平平安安回家才是王道啊以上是這樣的我今天晚餐時間使用UE叫了食物一般來說叫下去的一分鐘內就會收到金融visa卡的付款通知與電子郵件的下訂單通知但我這筆單沒有收到扣款通知甚至在取完餐之後在APP的查看電子明細也顯示未扣款致電去金融機構客服詢問也發現那筆金額並未送至金融機構想請問板上的大家有類似的經驗嗎我有點怕之後我之後被重複扣款或是繳其他項目的QQhttpsiimgurcomCP5kwaPjpg平常淋雨

In [7]:
ListA = "".join(List)
# print(ListA)
# 所有ptt外送版的文章 type為str
sub_ListA = re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a])","",ListA)
print(sub_ListA)# 所有ptt外送版的文章去標點、去多餘字詞

從歷史訂單也可連到之前訂過的店家點餐剛剛用電腦版的foodpanda有一陣子可以找店家訂餐不過現在不行了付款會失敗QQ去電腦版底下會有一排城市可以選選你在的城市就可以找到店家然後點餐了httpiimgurcomgLJ83UGjpg歷史訂單可訂熊貓復活了應該吧事實證明只是假象轉頭改訂Uber爛到炸裂不想給他賺了自從昨天被騙49訂閱之後我現在搜找不到餐廳404都來幹勒app恢復正常後好幾間我常點的店運費都從30調到60是怎樣圖你的地址沒有碼掉R你訂單有滿179嗎我是pro店家打折優惠放到購物車是原價跟客服反應說先訂看看若沒優惠再次反應超爛然後上週當機給的50折價券今天不能使用訂單的圖貼上來看看吧沒歷史訂單的圖po會員圖上來是要討論屁股訂單呢讓我看是不是沒179笑死自動續約日有時會遇到bug你是不是不知道怎麼碼掉地址看這麼多人問你都沒有回應原本那張圖金額5xx運費30印象中是這樣啦今天自動續約也是遇到太鳥了這系統謝謝告知避免大家吃bug自動續約也發生一樣問題你應該先問板主能不能發而不是用最後一句話當護身符連續三次自刪再貼幾近洗板不會用編輯功能補貨七五被改成只能在士東市場用了看一下說明謝謝告知這裡不是外送員板人氣也沒多高效果堪慮如果外送員會注意路況就不會當外送員了拉本來騎車就這種鳥樣才會這樣溫溫騎還不是會到幹嘛趕二樓不用一竿子打翻一船人今天過馬路才遇到一個闖紅燈的以為騎到待轉區偷偷轉過去就可以是不是UE今晚也發生付款失敗的情況便宜簡單夾鏈袋用安全帽藍牙芽耳機以前手機看影片過燙我會裝夾鏈袋泡水自製夾鏈袋雨天用個人蘋果I7有防潑水雨大會加個夾鏈袋及Airpods戴一耳沒有用過雨傘用過防水袋但跑長時間電量不夠加行動電源重耳機要找舒適的剛開始跑長時間還是會痛一個月後就習慣了大雨都爆單只按到接單跟取餐鍵跟開始導航在店家屋簷下完成這些步驟不用幾秒有時懶得手動會語音控制外送必備6號夾鏈袋motoA1迪卡儂有賣可觸控的手機防水袋手機觸控袋不錯我的是贈品所以有一個開口自己要再遮大部分都是夾鏈袋稱場手機防水袋的缺點是水氣問題還有需要拍照的時刻很麻煩第三是手機辨識螢幕關閉我自己有自製反光照剛好雨天也可以稱一下我從619開始訂熊貓外送也是一直信用卡刷卡被拒熊貓用之前用同一張信用卡都沒問題的說620在Ubereat用同一張信用卡也沒問題致電發卡銀行說是熊貓自己的問題居多熊貓客服回覆問題很慢都一直懷疑

In [8]:
all=sub_text+sub_ListA
all
# 文章留言全部相加

'熊貓掛了但是飢餓讓我找到了辦法httpsiimgurcombVlxLAGjpg1用電腦版登入2去google搜尋你要吃的店面的foodpanda網址httpsiimgurcomjGlftBojpg3正常點餐我猜app在已訂餐點按重新點餐也可以引述medama之銘言httpiimgurcomtGTWUlFjpg第一次看到這種情況如果下大雨就算了可是現在台北出大太陽耶而且才11點而已如題有訂閱pro卻還收取運費熊貓最近是怎麼了httpsiimgurcom0xqr49kjpg被收運費價錢會是28030因為買一送一httpsiimgurcomaYpxAMNjpg但我有訂閱各位好我是台北大學社會系張毓芬助理教授的學生這裡協助教授轉發誰是外送員台北北京武漢新加坡香港五個城市的比較研究的研究計畫目前台北組決定從大學生外送員開始做起我們正在尋找有外送工作經驗的受訪者不管是正在做的曾經做過的又願意接受訪談的都非常歡迎發信填寫表單報名我們會跟你聯繫以您方便的時間與地點進行訪談並提供全職外送員500車馬費兼職外送員300元車馬費如果你擔心疫情我們也可以電訪或依照你希望的方式進行訪談為了確保我們對你的談話內容有正確的理解我們會進行錄音並且會將錄音轉成逐字稿所有訪談內容百分之百保密使用資料時也會完全匿名謝謝大家疫情期間希望大家一切平安表單連結httpsbitly3gDwxuu如有違反板規還請版主提醒請問uber生鮮每週可以領取一次七五折的優惠碼那是我上週用完一次後這週領取新的也不能用是嗎剛剛用都無法使用不太懂所謂的一次是指永遠就這一次75折嗎謝謝答覆我知道外送員分秒必爭差了那幾分鐘的時間可能就會少好幾筆的單敬愛的外送員辛苦了重點我還是要提醒一下要趕單我知道也非常理解您們從商家出來開始跑單可以麻煩外送員也要注意一下路況今天差一點點就撞到外送員了從商家騎車硬切出來也沒注意左右來車這樣騎法真的很危險啊平平安安回家才是王道啊以上是這樣的我今天晚餐時間使用UE叫了食物一般來說叫下去的一分鐘內就會收到金融visa卡的付款通知與電子郵件的下訂單通知但我這筆單沒有收到扣款通知甚至在取完餐之後在APP的查看電子明細也顯示未扣款致電去金融機構客服詢問也發現那筆金額並未送至金融機構想請問板上的大家有類似的經驗嗎我有點怕之後我之後被重複扣款或是繳其他項目的QQhttpsiimgurcomCP5kwaPjpg平常淋

In [9]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import jieba
Text=all
# 默認模式
seg_list = jieba.cut(Text, cut_all=False)
print('generator: ',seg_list)
for seg in seg_list:
    print(seg,end=' ')
print('')
seg_list = jieba.lcut(Text, cut_all=False)
print('list: ',seg_list)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\USER\AppData\Local\Temp\jieba.cache


generator:  <generator object Tokenizer.cut at 0x000001DCF92D6F90>


Loading model cost 0.693 seconds.
Prefix dict has been built successfully.


熊貓掛 了 但是 飢餓 讓 我 找到 了 辦法 httpsiimgurcombVlxLAGjpg1 用 電腦版 登入 2 去 google 搜尋 你 要 吃 的 店面 的 foodpanda 網址 httpsiimgurcomjGlftBojpg3 正常 點餐 我 猜 app 在 已訂 餐點 按 重新 點餐 也 可以 引述 medama 之銘言 httpiimgurcomtGTWUlFjpg 第一次 看到 這種 情況 如果 下 大雨 就算 了 可是 現在 台北 出大太陽 耶 而且 才 11 點 而已 如題 有 訂閱 pro 卻還 收取 運費 熊貓 最近 是 怎麼 了 httpsiimgurcom0xqr49kjpg 被 收運費 價錢會 是 28030 因為 買 一送 一 httpsiimgurcomaYpxAMNjpg 但 我 有 訂閱 各位 好 我 是 台北 大學 社會系 張毓芬 助理 教授 的 學生 這裡 協助 教授 轉發 誰 是 外 送員 台北 北京 武漢 新加坡 香港 五個 城市 的 比 較 研究 的 研究 計畫 目前 台北 組決定 從 大學 生外 送員 開始 做起 我們 正在 尋找 有 外 送 工作 經驗 的 受 訪者 不管 是 正在 做 的 曾經 做過 的 又 願意 接受 訪談 的 都 非常 歡迎 發信 填 寫表單 報名 我們 會 跟 你 聯 繫 以 您 方便 的 時間 與 地點 進行訪 談並 提供 全職外 送員 500 車馬費 兼 職外 送員 300 元 車馬費 如果 你 擔心 疫情 我們 也 可以 電訪 或 依照 你 希望 的 方式 進行 訪談 為 了 確保 我們 對 你 的 談話 內容 有 正確 的 理解 我們 會 進行 錄音 並且 會將 錄音 轉成 逐字 稿 所有 訪談 內容 百分之百 保密 使用 資料 時 也 會 完全 匿名 謝謝 大家 疫情 期間 希望 大家 一切 平安 表單 連結 httpsbitly3gDwxuu 如有 違反板 規還 請 版主 提醒 請問 uber 生鮮 每週 可以 領取 一次 七五折 的 優惠碼 那 是 我 上 週用 完 一次 後 這週 領取 新 的 也 不能 用 是 嗎 剛剛 用 都 無法 使用 不太懂 所謂 的 一次 是 指永遠 就 這 一次 75 折 嗎 謝謝 答覆 我 知道 外送員 分秒 必爭 差 了 那 幾分鐘

才 會 知道 也 能 互相 體諒 而 不是 沒 任何 告知 狀況 下 還反 嗆 我們 然 後 做出 這麼 讓 人 難以 置信 的 行為 熊貓後續 處理 方式 真的 也 讓 人 非常 失望 應該 說 根本 完全 沒處理 httpsiimgurcomCjng0yUjpg 避免 影響 到 店家 所以 把 名字 遮掉 httpsiimgurcomDJy7Tmwjpg 雖然 說 不會 全部 外送員 都 是 這樣 畢竟 下 一單 的 外 送員態度 非常 好 但 外 送員素質 完全 不 篩選 發生 事情 處理 態度 真的 讓 人 很 失望 公告 foodpanda 最低 消費 金額 調整 通知 親愛的 foodpanda 用戶 你好 為 提供 更好 的 服務品 質自 202161 起 以下 指定 地區 桃園 市 苗栗 縣 臺 中市 彰化 縣 南投 縣雲林縣 嘉義縣 嘉義市 臺 南市 高雄市 屏東縣 宜蘭縣 花蓮縣 臺 東縣 澎湖 縣 美食 外送 與 生鮮 雜貨 部分 店家 最低 消費 金額 將調 整為 79 元 其他 一切 服務 皆 不受 影響 造成 您 的 不便 敬請 見諒 希望 未來 你 也 能 繼續 支持 我們 謝謝 祝 一切 順心 foodpanda 剛 收到 foodpanda 的 email 裡面 提到 低 消調 整到 79 元 之前 好像 是 69 所以 這樣 就算 加 最低 20 元運費 單筆 訂單 最少 也 要 花 99 元 了 我 今天 中午 收到 Foodpanda 寄 email 通知 訂閱 pandapro 自 今年 7 月 1 日要 開始 收費 價格 99 元 我 根本 只用 過 Foodpanda 外送 1 次 完全 不 知道 自己 有 訂閱 pandapro 怎麼 會 從 今年 7 月 1 日要 開始 向 我 收費 真的 是 覺得 很 莫名其妙 我該 如何 取消 訂閱 pandapro 我 是 絕對 不 可能 去 付費 訂閱 pandapro 不好意思 打擾 了 由 於 研究 的 用途 想 請問 各位 大大 能否 可以 得知 1 台南市 東區 一天 會 有 的 訂 單量 2 一個 時段 中 總共會 有 多少 個外 送員同 時 在 東區 送 餐在網 路上 找 了 好久 都 找 不到 資料 希望 有 相關 資訊 的 朋友 可以 與 我 分享 謝謝 大家

...

 都 來 幹勒 app 恢 復 正常 後 好 幾間 我常點 的 店 運費 都 從 30 調到 60 是 怎樣 圖 你 的 地址 沒有碼 掉 R 你訂 單有 滿 179 嗎 我 是 pro 店家 打折 優惠 放到 購物車 是 原價 跟 客服 反應 說 先訂 看看 若 沒 優惠 再次 反應 超爛 然後上 週當 機給 的 50 折價券 今天 不能 使用 訂單 的 圖 貼 上 來 看看 吧 沒 歷史 訂單 的 圖 po 會員圖 上 來 是 要 討論 屁股 訂單 呢 讓 我 看 是不是 沒 179 笑 死 自動續 約 日 有 時會 遇到 bug 你 是不是 不 知道 怎麼 碼掉 地址 看 這麼 多人問 你 都 沒 有 回應 原本 那張 圖金額 5xx 運費 30 印象 中是 這樣 啦 今天 自動續 約 也 是 遇到 太鳥 了 這系統 謝謝 告知 避免 大家 吃 bug 自動續 約 也 發生 一樣 問題 你 應該 先 問板 主能 不能 發而 不是 用 最 後 一句 話 當護身 符連續 三次 自刪 再 貼 幾近 洗板 不會用 編輯 功能 補貨 七五 被 改成 只能 在 士東 市場 用 了 看 一下 說明 謝謝 告知 這裡 不是 外送員板 人氣 也 沒多高 效果 堪慮 如果 外送員會 注意 路況 就 不會 當外 送員 了 拉本來 騎車 就 這種 鳥樣 才 會 這樣 溫溫 騎還 不是 會到 幹 嘛 趕二樓 不用 一竿子打 翻一船 人 今天 過馬路 才 遇到 一個 闖紅燈 的 以 為 騎 到 待 轉區 偷偷 轉過 去 就 可以 是不是 UE 今晚 也 發生 付款 失敗 的 情況 便宜 簡單 夾鏈 袋 用 安全帽 藍牙芽 耳機 以前 手機 看 影片 過燙 我會裝 夾鏈 袋 泡水 自 製 夾 鏈 袋 雨天 用個 人 蘋果 I7 有防潑 水雨大會 加個 夾鏈 袋 及 Airpods 戴一耳 沒 有用 過雨傘 用過 防水 袋 但 跑 長 時間 電量 不夠 加行動 電源 重耳 機要 找 舒適 的 剛開始 跑 長 時間 還是 會痛 一個 月 後 就 習慣 了 大雨 都 爆單 只 按 到 接單 跟 取 餐鍵 跟 開始 導航 在 店家 屋簷 下 完成 這些 步驟 不用 幾秒 有 時 懶 得手 動會 語音 控制 外送 必備 6 號夾 鏈袋 motoA1 迪卡儂 有 賣 可觸 控 的 手機 防水 袋手 機

 拿 去 吃 了 若 客服 還要 扣款 我 就算 了 當作 謝謝 辛苦 的 外 送員 了 樓 上 人 好好 我 的 超市 重 複 訂 單過 三小時 又 要 送 一次 幸好 熊貓 自己 取消 了 說 會 退款 但 還開 了 發票 來現 在 是不是 好 了 可以 訂 了 嗎 校正 回歸有 什麼 問題 嗎 外 送 問題 真的 太多貓 的 爛系 統用過 都 知道 爛 尤其 是 外 送端 熊貓 說 會 退 錢 的 聽 聽 就 好 自己 要 去 對 對 帳單 自己 討剛剛 看系統 我 昨天 送出 的 兩份 客服 處理 今天 竟然 消失 了 有人 也 消失 的 嗎 熊貓系 統出 包後來 有 公告 0 點到 5 點維修 昨晚 系統 異常下 單失敗 卻 多 了 三筆 歷史 訂單 使用 APP 送出 客訴 2 筆 今天 一 看客 訴 被 刪掉 同事 昨天 也 發生害 她 晚上 9 點在 那 邊問 誰 要 喝 熊貓 客服 還關 閉即 時 問答 真的 很 荒謬 多出 的 訂單 自己 取消 沒送 來 事後有 補發 50 元券給 我 不錯 昨天 熊貓 出包 今天 換 UE 還好 我 完美 的 避開 了 昨天 用 581 今天 用 熊貓 都 safeUE 若 是 昨天 之前 預約 的 不 影響 送餐 中 看來 預約 的 沒 影響加 11 我 的 熊貓 今天 也 是 一直 付款 失敗 主畫 面三槓 付款 方式 左滑 刪除 之前 ubereats 才 真的 叫 刪不掉 一直 跳針 說 錯誤 不給 刪 不死心 狂試 n 次 才 突然 成功 刪除 一樓 這個 跟 那個 不 太 一樣 那個 是 單純訂 單 付款 這是 會員 付款 我 的 主畫面 已經 沒有 付款 方式 選項 借問 有人 發現 現在 即使 是 pro 到 達 低消 卻 還是 被 收外 送費 嗎 樓 上 有 沒 有 截圖 啊 目前 沒發 生過 這種 事 通常 都 是 某個 沒 發現 的 折扣 造成 的 我 也 是 APP 刪不掉 但 用 電腦 網頁 版 的 話 就 可以 刪 除了 581 現在 全台 選項 只有 放在 門口 和 信用卡 交易 你 選門口 碰面 是 結 不了 帳的 系統 不會 讓 你 結帳 一定 要 改成 放在 門 口才 行感謝 回答 看來 三級 沒 解除 就 會 一直 持續 的 樣子 UE 手機版 強制 但是 電腦版 預設 為

 角度 如果 食物 能 搭配 飲料 一起 點 可以 省下 一筆外 送費 可以 增加 點這間 的 意願 但 多 數會 選擇會 分開 上架 這樣 兩間 店 曝光率 也 會 double 很多 店家 明明 就 一個 品牌 一個 店面 也 會 拆成 很多 店 上架 的 第一 點 可以 去 toberich 版問 看看 第二 點我身 為 客人 希望 一起 上 走 第一 條路 的 話 要 很 努力 透過 合 併 優勢 把 口碑 作起 來 有 口碑 才 有 提高 排序 跟 提高 外送 距離 的 可能 UE 部分 第二 種因 為 相 對門檻 低 也 是 為 什麼 多數 店家 會選擇 這樣 做 我 這邊 有 同 一家 店掛 三個 的 都 取 不同 名好 的 感謝 因 如果 分開 的 話 可能 月租 費 其他 的 都 要 有 兩個 以上 httpsiimgurcom9zumiaSjpg 剛 看到 這間 就是 二合一 我 是 消費者 有些 店 的 餐點 有 飲料 可以 另外 一起 搭配 的 話 會 想要 點 自己 有點過 幾家 有 時會 搭 著 飲料 一起 點單 單 只有 餐點 但 沒 有 其他 飲料 可 搭配 點的 話 就 不 一定 會 考慮點 那家 的 餐熊貓 客服 真的 很爛 所以 你 是 付 499 看 說 明要 折抵 後 599 啊 那 根本 是 機器 人 回覆 吧 當它 的 外 送員 也 沒 有 多 好 上次 大苑子 也 遇過 滿額 送 的 飲料 變成 要 錢 不過 回報 客服 直接 給我 120 券 已 移除 FOODPANDA 改 UBER 了 真心 爛 foodpro 一堆 bug 買 一送 一餐 點價格 全部 變兩倍 一樣 有 我 這個 月 剛加會員 優惠有 首購 五折 兩次 免運 一個 月 但 我 也 是 五月 新戶 不但 沒首購 五折 還被 收外 送費 ubereats 沒 客服 電話 可 打 寫 信去問 也 沒 下文 樓上 從 未點過 uber 的 新戶 認證 資格 很 嚴格 他 不是 新創 帳號 就是 新戶 新創 帳號 而且 要 輸入 完成 優惠碼 才 進行 點餐 才 算 新戶 如果 你 創 帳號 立刻 點餐 抱歉 你 就 沒 有 新戶 資格 所有 首次 的 都 不能 用樓 上 說 得 不大對 新戶 直接 點餐 沒輸入 折扣 碼一樣 有 免運 朋友 幾天前 開

 因此 不 一定 是 外 送員 故意 拖 時間 httpsiimgurcom6LfHPRqjpg 試試 沒 碰過 夾單 是 特定 區域 才 有 嗎 評分 低 於 90 分會 寄信 關注 再 沒 改進 就 會 停權 除非 情節 非常 嚴重 的 狀況 需要 用到 求助 內 的 選項 去 申訴 一般 的 負評 就 有 一定 的 效果 都市 傳說 負評 可以 降低 配對 到 的 機率 司機 其實 非常 在意 評分 的 上面 那篇 可以 參考 夾單 是 上 禮拜 公司 施行 的 策略 跟 熊貓 不 一樣 最 多只 會夾 一次 也 就是 取 取送 送 這 之中 只要 任何 一個 下 車點 拖 到 時間 都 會 影響 到後續 訂單 送 抵 的 時間 理論 上 沒 有司 機會 想 耽誤 時間 畢竟 配送 速度 最 直接 影響 收入 的 但 無法 控制 的 因素 包含 店家 拖餐 客戶 消失 等 風險 都 會 隨著 夾單 出現 增加 我 最近 訂 UE 也 遇到 司機會 在 中途 點取 餐 和 送餐 但 只要 不要 太誇張 的 送餐 時間 基本上 都 還是 會給 正評 所以 遇到 夾單 就 只能 希望 前 一位 訂餐 的 可以 準時 取餐 餐廳 也 能 準時 出餐 自動接 單然 後 在 路邊 打手 遊吧 這種 我 最怕 我 是 後 送 的 等 前 一個 愛拖時間 的 客人 或者 餐廳 等到 天荒地老 重開 重開 也 是 一樣 唷 用 網頁 版 登 看看 原來 還有 網頁 版轉 圈圈 中 QAQ 省錢版 有人 po 感覺 有點 寒酸 阿 巧克力 應該 比餅 乾貴 但 看 起來 很 不膨湃 三包 MM 巧克力 二條 TWIX 巧克力 一包 Airwaves 三張 十元 折價券 真的 百貨 公司 美食街 好像 很多 都 是 自己 開發票 的 不能 存載 具 推 希望 平台 開選項 可以 填發票 載 具給 自行 開電子 發票 的 店家 欸 欸 你 吃 的 怎麼 很 像 65 我 也 超常 叫 突然 想到 是 你 學校 離 我 公司 很近 XD 推彥 亭 兩邊 平台 都 有 送 啦 它 的 天 丼 不 錯 吃 福井 麵 疙瘩 也 ok 根本 隨叫 隨到 65 不行 啦 彥亭 福井 贏 百倍 鮭魚 雙享 是 爭鮮外 帶店 不是 迴 轉壽司 餐盒 原價 80httpstinyurlcom58qn7

list:  ['熊貓掛', '了', '但是', '飢餓', '讓', '我', '找到', '了', '辦法', 'httpsiimgurcombVlxLAGjpg1', '用', '電腦版', '登入', '2', '去', 'google', '搜尋', '你', '要', '吃', '的', '店面', '的', 'foodpanda', '網址', 'httpsiimgurcomjGlftBojpg3', '正常', '點餐', '我', '猜', 'app', '在', '已訂', '餐點', '按', '重新', '點餐', '也', '可以', '引述', 'medama', '之銘言', 'httpiimgurcomtGTWUlFjpg', '第一次', '看到', '這種', '情況', '如果', '下', '大雨', '就算', '了', '可是', '現在', '台北', '出大太陽', '耶', '而且', '才', '11', '點', '而已', '如題', '有', '訂閱', 'pro', '卻還', '收取', '運費', '熊貓', '最近', '是', '怎麼', '了', 'httpsiimgurcom0xqr49kjpg', '被', '收運費', '價錢會', '是', '28030', '因為', '買', '一送', '一', 'httpsiimgurcomaYpxAMNjpg', '但', '我', '有', '訂閱', '各位', '好', '我', '是', '台北', '大學', '社會系', '張毓芬', '助理', '教授', '的', '學生', '這裡', '協助', '教授', '轉發', '誰', '是', '外', '送員', '台北', '北京', '武漢', '新加坡', '香港', '五個', '城市', '的', '比', '較', '研究', '的', '研究', '計畫', '目前', '台北', '組決定', '從', '大學', '生外', '送員', '開始', '做起', '我們', '正在', '尋找', '有', '外', '送', '工作', '經驗', '的', '受', '訪者', '不管', '是', '正在', '做', '的', '曾經', '做過', '

In [3]:
import jieba
# jieba.set_dictionary('dict.txt.big')  # 如果是使用繁體文字，請記得去下載繁體字典來使用
with open('stops.txt', 'r', encoding='utf8') as f:  # 中文的停用字，我也忘記從哪裡拿到的，效果還可以，繁體字的資源真的比較少，大家將就一下吧
    stops = f.read().split('\n') 

In [4]:
stops.append('\n')  ## 我發現我的文章中有許多分行符號，這邊加入停用字中，可以把它拿掉
stops.append('\n\n')
terms = [t for t in jieba.cut(testStr, cut_all=True) if t not in stops]
sorted(Counter(terms).items(), key=lambda x:x[1], reverse=True)  ## 這個寫法很常出現在Ｃounter中，他可以排序，list每個item出現的次數。

﻿﻿,, , 、, 。, “, ”, 《, 》, ！, ，, ：, ；, ？, 人民, 末##末, 啊, 阿, 哎, 哎呀, 哎喲, 唉, 我, 我們, 按, 按照, 依照, 吧, 吧噠, 把, 罷了, 被, 本, 本著, 比, 比方, 比如, 鄙人, 彼, 彼此, 邊, 別, 別的, 別說, 並, 並且, 不比, 不成, 不單, 不但, 不獨, 不管, 不光, 不過, 不僅, 不拘, 不論, 不怕, 不然, 不如, 不特, 不惟, 不問, 不只, 朝, 朝著, 趁, 趁著, 乘, 沖, 除, 除此之外, 除非, 除了, 此, 此間, 此外, 從, 從而, 打, 待
但, 但是, 當, 當著, 到, 得, 的, 的話, 等, 等等, 地, 第, 叮咚, 對, 對於, 多, 多少, 而, 而況, 而且, 而是, 而外, 而言, 而已, 爾後, 反過來, 反過來說, 反之, 非但, 非徒, 否則, 嘎, 嘎登, 該, 趕, 個, 各, 各個, 各位, 各種, 各自, 給, 根據, 跟, 故, 故此, 固然, 關於, 管, 歸, 果然, 果真, 過, 哈, 哈哈, 呵, 和, 何, 何處, 何況, 何時, 嘿, 哼, 哼唷, 呼哧, 乎, 嘩, 還是, 還有, 換句話說, 換言之, 或, 或是, 或者, 極了, 及, 及其, 及至, 即, 即便
即或, 即令, 即若, 即使, 幾, 幾時, 己, 既, 既然, 既是, 繼而, 加之, 假如, 假若, 假使, 鑒於, 將, 較, 較之, 叫, 接著, 結果, 借, 緊接著, 進而, 盡, 儘管, 經, 經過, 就, 就是, 就是說, 據, 具體地說, 具體說來, 開始, 開外, 靠, 咳, 可, 可見, 可是, 可以, 況且, 啦, 來, 來著, 離, 例如, 哩, 連, 連同, 兩者, 了, 臨, 另, 另外, 另一方面, 論, 嘛, 嗎, 慢說, 漫說, 冒, 麼, 每, 每當, 們, 莫若, 某, 某個, 某些, 拿, 哪, 哪邊, 哪兒, 哪個, 哪裏, 哪年, 哪怕
哪天, 哪些, 哪樣, 那, 那邊, 那兒, 那個, 那會兒, 那裏, 那麼, 那麼些, 那麼樣, 那時, 那些, 那樣, 乃, 乃至, 呢, 能, 你, 你們, 您, 寧, 寧可, 寧肯, 寧願, 哦, 嘔, 啪達, 旁人, 呸, 憑, 憑藉, 其,

In [ ]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [13]:
lyrics = nltk.text.Text(jieba.cut(sub_ListA))
print(lyrics)

NameError: name 'nltk' is not defined

In [ ]:
# 統計詞頻

import pandas as pd
def count_segment_freq(seg_list):
    seg_df = pd.DataFrame(seg_list,columns=['seg'])
    seg_df['count'] = 1
    sef_freq = seg_df.groupby('seg')['count'].sum().sort_values(ascending=False)
    sef_freq = pd.DataFrame(sef_freq)
    return sef_freq
sef_freq = count_segment_freq(seg_list)
sef_freq.head(40)
# sef_freq.to_excel('text2.csv') 
#輸出Excel

In [ ]:
remove_words = [u'也',u',',u'很',u'∼',u'有',u'！',u'?',u'4',u'3',u'2',u'1',u'0',u'"',u'\n', u'.', u'的', u'，',u'但', u'是', u'會', u'對於', u'對',u'等',u'超',u'都',u'。',u' ',u'、',u'中',u'在',u'了',u'通常',u'如果',u'我們',u'需要'] 
# 自定義去除詞庫
seg_list_exact = jieba.cut(seg_list, cut_all = False) # 精確模式分詞
object_list = []
for word in seg_list: # 循環讀出每個分詞
    if word not in remove_words: # 如果不在去除詞庫中       
        object_list.append(word)
print(object_list)

In [ ]:
import pandas as pd
def count_segment_freq(seg_list):
    seg_df = pd.DataFrame(seg_list,columns=['seg'])
    seg_df['count'] = 1
    sef_freq = seg_df.groupby('seg')['count'].sum().sort_values(ascending=False)
    sef_freq = pd.DataFrame(sef_freq)
    return sef_freq
sef_freq = count_segment_freq(seg_list)
sef_freq.head(40)

In [ ]:
import pandas as pd
def count_segment_freq(List):
    seg_df = pd.DataFrame(List,columns=['seg'])
    seg_df['count'] = 1
    sef_freq = seg_df.groupby('seg')['count'].sum().sort_values(ascending=False)
    sef_freq = pd.DataFrame(sef_freq)
    return sef_freq
sef_freq = count_segment_freq(seg_list)
sef_freq.head(40)